In [75]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vehicle-sales-data/car_prices.csv


# Setup
Importing data

In [76]:
vehicle_location = '../input/vehicle-sales-data/car_prices.csv'
vehicle_data = pd.read_csv(vehicle_location)

# Showing table top 5 rows

In [92]:
vehicle_data.transmission.unique()

array(['automatic', nan, 'manual', 'Sedan', 'sedan'], dtype=object)

# Formating data

Spliting `saledate` to `sale_day`, `sale_month`, `sale_year` 

In [78]:
vehicle_data.dtypes

year              int64
make             object
model            object
trim             object
body             object
transmission     object
vin              object
state            object
condition       float64
odometer        float64
color            object
interior         object
seller           object
mmr             float64
sellingprice    float64
saledate         object
dtype: object

In [79]:
vehicle_data_formated = vehicle_data
vehicle_data_formated.drop(labels = ['interior','seller'],axis = 1,inplace = True)
vehicle_data_formated['saledate'] = pd.to_datetime(vehicle_data['saledate'], errors = 'coerce')

/tmp/ipykernel_33/1484293596.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  vehicle_data_formated['saledate'] = pd.to_datetime(vehicle_data['saledate'], errors = 'coerce')
/tmp/ipykernel_33/1484293596.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  vehicle_data_formated['saledate'] = pd.to_datetime(vehicle_data['saledate'], errors = 'coerce')


## Hecking errors

In [80]:
vehicle_data_formated[vehicle_data['saledate'].isnull()].head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,mmr,sellingprice,saledate
306447,2013,Hyundai,Sonata,GLS,Sedan,automatic,5npeb4ac6dh687932,il,19.0,37254.0,gray,NaN,NaN,NaN
406524,2013,Chrysler,300,Base,Sedan,automatic,2c3ccaag9dh723146,il,27.0,44208.0,silver,NaN,NaN,NaN
408161,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj4fm201708,NaN,46.0,4802,NaN,13200.0,NaT
417835,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj2fm258506,NaN,1.0,9410,NaN,13300.0,NaT
421289,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,automatic,3vwd17aj3fm276741,NaN,46.0,1167,NaN,13200.0,NaT


My thought about `NaN` value is that these cars have not been sold yet. \
I don't delete records because I can use them in the future to analyze the least selling cars. \
\
I try to change cells because they are mixed between columnss.

In [81]:
#function to find transmission in columns 
def find_transmission(row):
    for record in row:
        if record == 'automatic' or record == 'manual':
            return record
    return row['transmission']
#apply function and changing data
vehicle_data_formated['transmission'] = vehicle_data.apply(find_transmission, axis=1)
#checking what left
vehicle_data_formated[(vehicle_data_formated['transmission'] != 'automatic') & (vehicle_data_formated['transmission'] != 'manual') & (
vehicle_data_formated['transmission'].isna() !=True)]

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,mmr,sellingprice,saledate
461612,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,NaN,3vwd17aj3fm259017,NaN,46.0,2711,NaN,14250.0,NaT
505299,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,NaN,3vwd17aj7fm222388,NaN,36.0,20379,NaN,13600.0,NaT
529009,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,NaN,3vwd17aj8fm298895,NaN,2.0,2817,NaN,13750.0,NaT
551222,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,NaN,3vwd17aj8fm239622,NaN,2.0,9562,NaN,13200.0,NaT


In [82]:
def find_vin(row):
    for record in row:
        try:
            if len(record) == 17:
                return record
        except:
            pass
    return row['vin']
vehicle_data_formated['vin'] = vehicle_data.apply(find_vin, axis = 1)

In [83]:
def find_state(row):
    for record in row:
        try:
            if len(record) == 2 and record.islower():
                return record
        except:
            pass
    return row['state']
vehicle_data_formated['state'] = vehicle_data.apply(find_state, axis = 1)

In [84]:
vehicle_data_formated.sample(5)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,mmr,sellingprice,saledate
141961,2013,Chrysler,200,Touring,Sedan,automatic,1c3ccbbgxdn744437,tx,29.0,21003.0,red,12400.0,10600.0,2015-01-27 02:30:00+08:00
178670,2004,Lexus,RX 330,Base,SUV,automatic,2t2ga31u64c006441,ga,36.0,75007.0,blue,9375.0,11900.0,2015-01-22 02:00:00+08:00
117101,2006,Ford,Econoline Cargo,E-350 Super Duty,Van,automatic,1ftse34l36hb38190,wi,19.0,160718.0,white,4050.0,4000.0,2015-01-28 02:00:00+08:00
156110,2014,Ford,E-Series Van,E-250,E-Series Van,NaN,1ftne2ew7eda31519,tx,21.0,8765.0,white,22300.0,21200.0,2015-02-18 02:00:00+08:00
415690,2014,Buick,Enclave,Leather Group,SUV,automatic,5gakrbkd4ej115956,fl,47.0,33812.0,silver,27300.0,29800.0,2015-05-26 02:30:00+07:00


## Formating car makers

In [94]:
vehicle_data_formated = vehicle_data_formated[vehicle_data_formated['make'].isnull() == False]
car_makers = vehicle_data_formated.make.unique()
np.unique(np.array([car.lower() for car in car_makers]))

array(['acura', 'airstream', 'aston martin', 'audi', 'bentley', 'bmw',
       'buick', 'cadillac', 'chevrolet', 'chrysler', 'daewoo', 'dodge',
       'ferrari', 'fiat', 'fisker', 'ford', 'ford tk', 'ford truck',
       'geo', 'gmc', 'honda', 'hummer', 'hyundai', 'infiniti', 'isuzu',
       'jaguar', 'jeep', 'kia', 'lamborghini', 'landrover', 'lexus',
       'lincoln', 'lotus', 'maserati', 'mazda', 'mercedes', 'mercury',
       'mini', 'mitsubishi', 'nissan', 'oldsmobile', 'plymouth',
       'pontiac', 'porsche', 'ram', 'rolls-royce', 'saab', 'saturn',
       'scion', 'smart', 'subaru', 'suzuki', 'tesla', 'toyota',
       'volkswagen', 'volvo'], dtype='<U12')

### Hanging car makers 

In [93]:
def formating_car_makers(brand):
    if isinstance(brand, str):
        mapping = {
            'chev truck': 'chevrolet',
            'dodge tk': 'dodge',
            'gmc truck': 'gmc',
            'hyundai tk': 'hyundai',
            'land rover': 'landrover',
            'mazda tk': 'mazda',
            'mercedes-b': 'mercedes',
            'mercedes-benz': 'mercedes',
            'vw': 'volkswagen'
        }
        brand = brand.lower()
        if brand in mapping:
            return mapping[brand]
        else:
            return brand
    else:
        ''
    
vehicle_data_formated.make = vehicle_data.make.apply(lambda brand: formating_car_makers(brand))
vehicle_data_formated = vehicle_data_formated[vehicle_data_formated['make'] != 'dot']

## Formating car models

In [171]:
car_model = vehicle_data_formated.model.unique()
grouped['count'] = vehicle_data_formated.groupby('model')['make'].transform('count')
grouped = grouped[grouped['count'] <= 5]
grouped

/tmp/ipykernel_33/3311932684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped['count'] = vehicle_data_formated.groupby('model')['make'].transform('count')


,make,model,count
40,cadillac,ELR,5.0
2584,bentley,Continental GT Speed,3.0
2719,pontiac,g6,4.0
3236,bmw,7,2.0
3483,mitsubishi,lancer,4.0
...,...,...,...
475043,audi,Q3,1.0
529862,ferrari,360,1.0
539490,mercedes,GLA-Class,2.0
547702,acura,TLX,1.0
